In [48]:
%cd /home/druhe/rail1/src/cnf
%load_ext autoreload
%autoreload 2

import os
os.environ['DATAROOT'] = '/home/druhe/datasets'


/home/druhe/rail1/src/cnf
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


/home/druhe/rail1/src/cnf/.venv/lib/python3.10/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [49]:
import wandb
import rail1.run
import rail1.checkpoint
import rail1.utils
import tempfile
import models
import datasets
import torch
import torch.nn.functional as F
import re

torch.set_grad_enabled(False)
rail1.utils.set_seed(0, deterministic=False)

In [50]:
api = wandb.Api()
path = 'druhe/cnf/hiq9zm8b'
run = api.run(path)
run_dir = os.path.join('runs', 'notebooks', run.id)
os.makedirs(run_dir, exist_ok=True)

In [51]:

artifact_dict = {}

for a in run.logged_artifacts():

    if a.type != "checkpoint":
        continue

    file = a.metadata['filename']

    metrics = file.rsplit('/')[-1]

    match = re.search(r"-val_neg_iou=(-\d+\.\d+)", metrics)

    neg_iou = float(match.group(1)) if match else None

    dictionary = {'neg_iou': neg_iou}

    dictionary['artifact'] = a

    artifact_dict[a] = dictionary


In [52]:
artifact_dict = {k: v for k, v in artifact_dict.items() if v['neg_iou'] is not None}

In [53]:
best_artifact = min(artifact_dict, key=lambda x: artifact_dict[x]['neg_iou'])

In [54]:
best_artifact.download(root=os.path.join(run_dir, 'checkpoints'))

wandb:   1 of 1 files downloaded.  
wandb: WARNING Source type is set to 'repo' but some required information is missing from the environment. A job will not be created from this run. See https://docs.wandb.ai/guides/launch/create-job


'runs/notebooks/hiq9zm8b/checkpoints'

In [55]:
config = run.config

In [56]:
model = getattr(models, config["model"].pop("name"))(**config["model"])




In [57]:
train_state = {}
optimizer = None
rail1.checkpoint.load_checkpoint(os.path.join(run_dir, 'checkpoints'), model, train_state=train_state, optimizer=optimizer)


Successfully restored complete state from: runs/notebooks/hiq9zm8b/checkpoints/step=77824-epoch=0-val_loss=15.7940-val_neg_iou=-0.9304-val_s_it=0.0249.pt



{'global_step': 77824,
 'last_global_step': 77824,
 'should_raise': None,
 'current_epoch': 0,
 'device': device(type='cuda', index=0),
 'train_metrics': defaultdict(list,
             {'loss': [tensor([16.1938, 13.6535, 17.8294, 18.0112, 15.9553, 14.8408, 10.8084, 21.1689,
                       11.5269, 10.3226, 15.3130, 12.8669, 10.6736, 12.6412,  9.1542, 11.2532],
                      device='cuda:0'),
               tensor([ 8.7740,  8.1999, 22.9469, 23.3657,  8.6043, 26.4167, 10.0762, 12.9795,
                       13.7604, 20.2086,  9.9927, 27.4175, 25.3610, 15.3054, 12.0219, 13.4289],
                      device='cuda:0'),
               tensor([11.0404, 30.1185, 15.7833,  8.8691,  7.5659,  9.6644,  8.2191,  9.6064,
                       11.0079,  8.1241, 20.0432,  9.3383, 10.7815, 18.7779, 11.4723, 27.7128],
                      device='cuda:0'),
               tensor([15.1763, 21.0724, 11.4410, 20.2192, 13.4554, 11.9887, 10.9133, 25.9882,
                       12.8582, 

In [58]:
dataset_config = {'name': 'modelnet40'}


In [59]:
def modelnet_files(path):

    num_classes = 40

    def fetch_files(filelist):
        return [item.strip() for item in open(filelist).readlines()]

    train_files = fetch_files(os.path.join(path, 'train_files.txt'))
    test_files = fetch_files(os.path.join(path, 'test_files.txt'))

    return num_classes, train_files, test_files


In [60]:
import os, sys, torch, h5py, warnings, numpy as np
import open3d as o3d
import torch.utils.data

class H5PyDataset(torch.utils.data.Dataset):
    def __init__(self, file_list, num_point=2048, data_aug=0):
        self.num_point = num_point
        self.file_list = file_list
        self.points_list = np.zeros((1, num_point, 3))
        self.labels_list = np.zeros((1,))
        self.data_aug = data_aug

        for file in self.file_list:
            data, label = self.loadh5DataFile(file)
            self.points_list = np.concatenate(
                [self.points_list, data[:, :self.num_point, :]], axis=0)
            self.labels_list = np.concatenate([self.labels_list, label.ravel()], axis=0)
        
        self.points_list = self.points_list[1:]
        self.labels_list = self.labels_list[1:]
        assert len(self.points_list) == len(self.labels_list)
        print('Number of Objects: ', len(self.labels_list))

    @staticmethod
    def loadh5DataFile(PathtoFile):
        f = h5py.File(PathtoFile, 'r')
        return f['data'][:], f['label'][:]

    def __len__(self):
        return len(self.points_list)

    def __getitem__(self, index):
        point_xyz = self.points_list[index][:, 0:3]

        point_label = self.labels_list[index].astype(np.int32)

        return point_xyz, point_label

In [61]:
_, train_files, test_files = modelnet_files(os.path.join(os.environ['DATAROOT'], 'modelnet40_ply_hdf5_2048'))

train_files = [os.path.join(os.environ['DATAROOT'], 'modelnet40_ply_hdf5_2048', file.split('/')[-1]) for file in train_files]
test_files = [os.path.join(os.environ['DATAROOT'], 'modelnet40_ply_hdf5_2048', file.split('/')[-1]) for file in test_files]


train_dataset = H5PyDataset(train_files, num_point=2048)
test_dataset = H5PyDataset(test_files, num_point=2048)


Number of Objects:  9840
Number of Objects:  2468


In [62]:
from rail1.data import batchloader

train_loader = batchloader.BatchLoader(train_dataset, batch_size=24, shuffle=False, num_workers=4)
test_loader = batchloader.BatchLoader(test_dataset, batch_size=24, shuffle=False, num_workers=4)




In [63]:
model = model.eval().cuda()

In [64]:
from models.modules import dgcnn_cls


def encode_fn(model, x):

    x = x.transpose(2, 1).contiguous()

    batch_size, _, num_points = x.size()
    x = dgcnn_cls.get_graph_feature(
        x, k=model.k
    )  # (batch_size, 3, num_points) -> (batch_size, 3*2, num_points, k)
    x = model.conv1(
        x
    )  # (batch_size, 3*2, num_points, k) -> (batch_size, 64, num_points, k)
    x1 = x.max(dim=-1, keepdim=False)[
        0
    ]  # (batch_size, 64, num_points, k) -> (batch_size, 64, num_points)

    x = dgcnn_cls.get_graph_feature(
        x1, k=model.k
    )  # (batch_size, 64, num_points) -> (batch_size, 64*2, num_points, k)
    x = model.conv2(
        x
    )  # (batch_size, 64*2, num_points, k) -> (batch_size, 64, num_points, k)
    x2 = x.max(dim=-1, keepdim=False)[
        0
    ]  # (batch_size, 64, num_points, k) -> (batch_size, 64, num_points)

    x = dgcnn_cls.get_graph_feature(
        x2, k=model.k
    )  # (batch_size, 64, num_points) -> (batch_size, 64*2, num_points, k)
    x = model.conv3(
        x
    )  # (batch_size, 64*2, num_points, k) -> (batch_size, 128, num_points, k)
    x3 = x.max(dim=-1, keepdim=False)[
        0
    ]  # (batch_size, 128, num_points, k) -> (batch_size, 128, num_points)

    x = dgcnn_cls.get_graph_feature(
        x3, k=model.k
    )  # (batch_size, 128, num_points) -> (batch_size, 128*2, num_points, k)
    x = model.conv4(
        x
    )  # (batch_size, 128*2, num_points, k) -> (batch_size, 256, num_points, k)
    x4 = x.max(dim=-1, keepdim=False)[
        0
    ]  # (batch_size, 256, num_points, k) -> (batch_size, 256, num_points)

    x = torch.cat((x1, x2, x3, x4), dim=1)  # (batch_size, 512, num_points)

    x = model.conv5(
        x
    )  # (batch_size, 512, num_points) -> (batch_size, feat_dims, num_points)
    x = x.max(dim=-1, keepdim=True)[
        0
    ]  # (batch_size, feat_dims, num_points) -> (batch_size, feat_dims)

    return x

In [65]:
from rail1.utils import math as math_utils
X_train, y_train = [], []


for i in range(len(train_loader)):

    points, target = train_loader[i]

    if points.shape[2] == 6:
        points = points[:, :, :3]

    points, target = points.float().cuda(), target.long().cuda()

    # z = model.encoder.dgcnn_encoder(points)
    z = encode_fn(model.encoder.dgcnn_encoder, points)
    X_train.append(z.cpu().detach().numpy())
    y_train.append(target.cpu().detach().numpy())
    

X_train = np.concatenate(X_train, axis=0)
y_train = np.concatenate(y_train, axis=0)


print("Done!")

Done!


In [66]:
X_test, y_test = [], []

num_test_batches = math_utils.ceildiv(
    len(test_loader.dataset), test_loader.batch_size
)

for i in range(len(test_loader)):

        points, target = test_loader[i]
    
        if points.shape[2] == 6:
            points = points[:, :, :3]
    
        points, target = points.float().cuda(), target.long().cuda()
    
        z = encode_fn(model.encoder.dgcnn_encoder, points)
        X_test.append(z.cpu().detach().numpy())
        y_test.append(target.cpu().detach().numpy())

X_test = np.concatenate(X_test, axis=0)
y_test = np.concatenate(y_test, axis=0)

print("Done!")


Done!


In [67]:
X_train = X_train.squeeze(-1)
X_test = X_test.squeeze(-1)

In [68]:
X_train.max()

10.693303

In [69]:
X_train.min()


-0.61787605

In [72]:
from sklearn import svm, metrics
# Import standard scaler
from sklearn.preprocessing import MinMaxScaler, StandardScaler

scaler = MinMaxScaler(feature_range=(-1, 1))
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

clf = svm.LinearSVC(random_state=0, max_iter=1000, dual='auto', C=0.01)
clf.fit(X_train, y_train)
result = clf.predict(X_test)  
accuracy = np.sum(result==y_test).astype(float) / np.size(y_test)
print("Transfer linear SVM accuracy: {:.2f}%".format(accuracy*100))


Transfer linear SVM accuracy: 91.00%


In [44]:
X_train.min(), X_train.max()

(-1.0, 1.0)

In [37]:
X_train.shape

(9840, 1024)